Análisis y Predicción de Precios de Autos (Core)

Objetivo: 
 Aplicar técnicas de regresión lineal, KNN y árboles de decisión para predecir precios de autos utilizando un dataset de Kaggle. Realizar un Análisis Exploratorio de Datos (EDA) completo y comparar el rendimiento de los modelos mediante benchmarking.

# Parte 1: Análisis Exploratorio de Datos (EDA)

##  1. Carga y Exploración de Datos:

* Cargar el dataset y revisar la estructura básica.
* Descripción de las variables y su distribución.
* Detección y tratamiento de valores nulos.
* Identificación y tratamiento de outliers.
* Análisis de correlación entre variables.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# cargar el dataset automobile data
data = pd.read_csv('../data/Automobile_data.csv')
print(data.shape)
print(data.columns)
data.info()
data.head(5)

(205, 26)
Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 no

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [42]:
print(data.describe())
print("Valores faltantes:", data.isnull().sum())

        symboling  wheel-base      length       width      height  \
count  205.000000  205.000000  205.000000  205.000000  205.000000   
mean     0.834146   98.756585  174.049268   65.907805   53.724878   
std      1.245307    6.021776   12.337289    2.145204    2.443522   
min     -2.000000   86.600000  141.100000   60.300000   47.800000   
25%      0.000000   94.500000  166.300000   64.100000   52.000000   
50%      1.000000   97.000000  173.200000   65.500000   54.100000   
75%      2.000000  102.400000  183.100000   66.900000   55.500000   
max      3.000000  120.900000  208.100000   72.300000   59.800000   

       curb-weight  engine-size  compression-ratio    city-mpg  highway-mpg  
count   205.000000   205.000000         205.000000  205.000000   205.000000  
mean   2555.565854   126.907317          10.142537   25.219512    30.751220  
std     520.680204    41.642693           3.972040    6.542142     6.886443  
min    1488.000000    61.000000           7.000000   13.000000    

In [ ]:
data_type = {
    'make' : 'category',
    'num-of-doors' : 'category',
    'horsepower' : 'int64',
    'price' : 'float64',
    'drive-wheels' : 'category',
}
data = data.astype(data_type)



### Algunas columnas traen el simbolo "?"

In [4]:
# Lista de columnas con valores problemáticos
columns_with_issues = ['horsepower', 'engine-size']

# Reemplaza '?' con NaN y convierte a numérico, forzando valores no convertibles a NaN
for col in columns_with_issues:
    data[col] = data[col].replace('?', np.nan)
    data[col] = pd.to_numeric(data[col], errors='coerce')
imputer = SimpleImputer(strategy='mean')
data[columns_with_issues] = imputer.fit_transform(data[columns_with_issues])

# Reemplaza '?' con NaN en la columna 'price' (si es que aparece)
data['price'] = data['price'].replace('?', np.nan)

# Elimina las filas donde 'price' es NaN (es decir, donde originalmente había '?')
data = data.dropna(subset=['price'])

data_type = {
    'make' : 'category',
    'aspiration' : 'category',
    'drive-wheels' : 'category',
    'horsepower' : 'float64',
    'price' : 'float64'
}
data = data.astype(data_type)
print(data.isnull().sum())


symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64


In [ ]:
print(data.describe())

# Parte 2: Preparación de Datos

##  2. Preprocesamiento:

* Selección de características importantes.
* Transformación de variables categóricas.
* División del conjunto de datos en entrenamiento y prueba.
* Escalado de características.

In [10]:
# 1. Preparar los datos
X = data[['horsepower', 'engine-size', 'make', 'drive-wheels', 'aspiration']]
y = data['price']

# 2. Convertir las variables categóricas a variables dummy (OneHotEncoding)
X = pd.get_dummies(X, drop_first=True)  # Esto crea variables dummy para la columna 'make'

# 3. Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Parte 3: Modelos de Regresión

## Regresion lineal

In [11]:
# Crear el modelo de regresión lineal
model = LinearRegression()

# Ajustar el modelo (entrenarlo)
model.fit(X_train, y_train)

# Hacer predicciones con el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
lmae = mean_absolute_error(y_test, y_pred)
lmse = mean_squared_error(y_test, y_pred)
lr2 = r2_score(y_test, y_pred)

# Imprimir los resultados
print(f"Mean Absolute Error (MAE): {lmae:.2f}")
print(f"Mean Squared Error (MSE): {lmse:.2f}")
print(f"R-squared (R²): {lr2:.2f}")

Mean Absolute Error (MAE): 2111.65
Mean Squared Error (MSE): 11340130.14
R-squared (R²): 0.91


## K-Nearest Neighbors (KNN):

Entrenamiento del modelo.
Evaluación del rendimiento (MSE y R²).


In [12]:
# 1. Preparar los datos
X = data[['horsepower', 'engine-size', 'make', 'drive-wheels', 'aspiration']]
y = data['price']

# 2. Definir columnas numéricas y categóricas
numeric_features = ['horsepower', 'engine-size']
categorical_features = ['make', 'drive-wheels', 'aspiration']

# 3. Crear un transformador de columnas para escalar numéricas y OneHotEncode en categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# 4. Crear un pipeline para escalar, codificar y aplicar KNN
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsRegressor(n_neighbors=5))
])

# 5. Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Entrenar el modelo
model.fit(X_train, y_train)

# 7. Hacer predicciones
y_pred = model.predict(X_test)

# 8. Evaluar el modelo
kmae = mean_absolute_error(y_test, y_pred)
kmse = mean_squared_error(y_test, y_pred)
kr2 = r2_score(y_test, y_pred)

# 9. Imprimir los resultados
print(f"Mean Absolute Error (MAE): {kmae:.2f}")
print(f"Mean Squared Error (MSE): {kmse:.2f}")
print(f"R-squared (R²): {kr2:.2f}")


Mean Absolute Error (MAE): 3139.46
Mean Squared Error (MSE): 27941089.98
R-squared (R²): 0.77


## Árbol de Decisión:

Entrenamiento del modelo.
Evaluación del rendimiento (MSE y R²).


In [14]:
# Preparar los datos
X = data[['horsepower', 'engine-size', 'make', 'drive-wheels', 'aspiration']]
y = data['price']

# Definir columnas numéricas y categóricas
numeric_features = ['horsepower', 'engine-size']
categorical_features = ['make', 'drive-wheels', 'aspiration']

# Crear un transformador de columnas solo para OneHotEncode en categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ],
    remainder='passthrough'  # Mantener las columnas numéricas sin transformar
)

# Crear un pipeline para codificar y aplicar el árbol de decisión
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeRegressor(random_state=42))
])

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
amae = mean_absolute_error(y_test, y_pred)
amse = mean_squared_error(y_test, y_pred)
ar2 = r2_score(y_test, y_pred)

# Imprimir los resultados
print(f"Mean Absolute Error (MAE): {amae:.2f}")
print(f"Mean Squared Error (MSE): {amse:.2f}")
print(f"R-squared (R²): {ar2:.2f}")

Mean Absolute Error (MAE): 2038.78
Mean Squared Error (MSE): 8363499.82
R-squared (R²): 0.93


# Parte 4: Benchmarking y Comparación de Modelos

 ## Comparación de Modelos:

Comparar los resultados de los tres modelos en términos de MSE y R².
Discusión sobre las diferencias en el rendimiento de los modelos.



In [15]:

results = {
    'Modelo': ['Regresión Lineal', 'K-Nearest Neighbors', 'Árbol de Decisión'],
    'MAE': [lmae, kmae, amae],
    'MSE': [lmse, kmse, amse],
    'R²': [lr2, kr2, ar2]
}

# Crear un DataFrame para visualizar los resultados
results_df = pd.DataFrame(results)
print(results_df)

                Modelo          MAE           MSE        R²
0     Regresión Lineal  2111.650058  1.134013e+07  0.907312
1  K-Nearest Neighbors  3139.463415  2.794109e+07  0.771624
2    Árbol de Decisión  2038.782811  8.363500e+06  0.931641


# Conclusion:
Claramente el modelo de Arbol de decision tiene un mejor resultado de prediccion al tener un r2 muy cercano al 1, ademas, tiene el MSE y el MAE mas bajo entre los 3 modelos, lo que indica que el modelo de Arbol de decision es el mas preciso y efectivo para este trabajo